In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
# โหลดข้อมูลจากไฟล์ CSV
file_name = 'weather_100_years_extended (1).csv'
data = pd.read_csv(file_name)

In [ ]:
# แสดงข้อมูลเบื้องต้น
print("ข้อมูลเบื้องต้น:\n", data.head())
print("ข้อมูลที่หายไปในแต่ละคอลัมน์:\n", data.isnull().sum())


ข้อมูลเบื้องต้น:
          date   country     city  temperature (°C)  humidity (%)  \
0  1925-01-01  Thailand  Bangkok         28.488867     63.548650   
1  1925-01-02  Thailand  Bangkok         28.992703     95.005496   
2  1925-01-03  Thailand  Bangkok         21.070264     99.813784   
3  1925-01-04  Thailand  Bangkok         32.380113     76.746228   
4  1925-01-05  Thailand  Bangkok               NaN     61.691885   

   wind_speed (km/h)  
0          29.789863  
1          25.575581  
2           1.543178  
3           4.649422  
4          31.451403  
ข้อมูลที่หายไปในแต่ละคอลัมน์:
 date                    0
country                 0
city                    0
temperature (°C)     8528
humidity (%)         8549
wind_speed (km/h)    8473
dtype: int64


In [ ]:
# ตรวจสอบคอลัมน์ที่ต้องการ
required_columns = ['date', 'country', 'city', 'temperature (°C)', 'humidity (%)', 'wind_speed (km/h)']
for col in required_columns:
    if col not in data.columns:
        raise ValueError(f"Missing required column: {col}")

In [ ]:
# ลบคอลัมน์ที่ไม่จำเป็นสำหรับการพยากรณ์ (date, country, city)
X = data.drop(columns=['date'])
y_temp = data['temperature (°C)'].ffill()

# แปลงข้อมูล 'country' เป็นตัวเลข (One-Hot Encoding)
data = pd.get_dummies(data, columns=['country', 'city'], drop_first=True)

# ตรวจสอบว่ามีคอลัมน์ใหม่ที่ถูกสร้างขึ้น
print("คอลัมน์ที่ได้จาก One-Hot Encoding:\n", data.columns)

# แปลงค่า True/False ให้เป็น 0/1 (ถ้ามี)
bool_columns = data.select_dtypes(include=['bool']).columns
data[bool_columns] = data[bool_columns].astype(int)

# แสดงตัวอย่างข้อมูลที่แปลงแล้ว
print(data.head())


คอลัมน์ที่ได้จาก One-Hot Encoding:
 Index(['date', 'temperature (°C)', 'humidity (%)', 'wind_speed (km/h)',
       'country_France', 'country_Japan', 'country_South Korea',
       'country_Thailand', 'country_UK', 'country_USA', 'city_Beijing',
       'city_London', 'city_New York', 'city_Paris', 'city_Seoul',
       'city_Tokyo'],
      dtype='object')
         date  temperature (°C)  humidity (%)  wind_speed (km/h)  \
0  1925-01-01         28.488867     63.548650          29.789863   
1  1925-01-02         28.992703     95.005496          25.575581   
2  1925-01-03         21.070264     99.813784           1.543178   
3  1925-01-04         32.380113     76.746228           4.649422   
4  1925-01-05               NaN     61.691885          31.451403   

   country_France  country_Japan  country_South Korea  country_Thailand  \
0               0              0                    0                 1   
1               0              0                    0                 1   
2         

In [ ]:
print(X.head())  # ตรวจสอบข้อมูล X
print(X.isnull().sum())  # ตรวจสอบค่าที่หายไป


    country     city  temperature (°C)  humidity (%)  wind_speed (km/h)
0  Thailand  Bangkok         28.488867     63.548650          29.789863
1  Thailand  Bangkok         28.992703     95.005496          25.575581
2  Thailand  Bangkok         21.070264     99.813784           1.543178
3  Thailand  Bangkok         32.380113     76.746228           4.649422
4  Thailand  Bangkok               NaN     61.691885          31.451403
country                 0
city                    0
temperature (°C)     8528
humidity (%)         8549
wind_speed (km/h)    8473
dtype: int64


In [ ]:
# แยกข้อมูลตัวเลขและข้อมูลหมวดหมู่
numeric_cols = X.select_dtypes(include=['number']).columns  # เลือกเฉพาะคอลัมน์ที่เป็นตัวเลข
categorical_cols = X.select_dtypes(exclude=['number']).columns  # เลือกเฉพาะคอลัมน์ที่เป็นข้อความ

# เติมค่าที่หายไปในข้อมูลตัวเลขด้วยค่าเฉลี่ย (mean)
num_imputer = SimpleImputer(strategy='mean')
X_numeric_imputed = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols)

# เติมค่าที่หายไปในข้อมูลหมวดหมู่ด้วยค่าที่พบบ่อยที่สุด (most_frequent)
cat_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_imputed = pd.DataFrame(cat_imputer.fit_transform(X[categorical_cols]), columns=categorical_cols)

# รวมข้อมูลทั้งสองประเภทกลับมา
X_imputed = pd.concat([X_numeric_imputed, X_categorical_imputed], axis=1)

# แสดงข้อมูลที่เติมค่ากลับมา
missing_values = X_imputed.isnull().sum()
print(missing_values)



temperature (°C)     0
humidity (%)         0
wind_speed (km/h)    0
country              0
city                 0
dtype: int64


In [ ]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed[numeric_cols]), columns=numeric_cols)


In [ ]:
X_train, X_test, y_train_temp, y_test_temp = train_test_split(
    X_scaled, y_temp, train_size=0.2, random_state=42
)




In [ ]:
# สร้างโมเดล RandomForest
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb.fit(X_train, y_train_temp)

y_pred_xgb = xgb.predict(X_test)


#decision_tree = DecisionTreeRegressor(random_state=42)
svm = SVR(kernel='rbf', C=5.0, epsilon=0.05)

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

#elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)  # alpha: ค่าการปรับ, l1_ratio: สัดส่วนของ L1 กับ L2

In [ ]:
# ฝึกสอนโมเดลทั้งหมด
xgb.fit(X_train, y_train_temp)
rf.fit(X_train, y_train_temp)
svm.fit(X_train, y_train_temp)


SVR(C=5.0, epsilon=0.05)

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    accuracy = 100 - mape

    print(f"{model.__class__.__name__} Evaluation")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R^2 Score: {r2}")
    print(f"MAPE: {mape}%")
    print(f"Prediction Accuracy: {accuracy}%")
    print("=" * 50)
    return y_pred


In [ ]:
# Evaluate Models
evaluate_model(xgb, X_test, y_test_temp)
evaluate_model(rf, X_test, y_test_temp)
evaluate_model(svm, X_test, y_test_temp)



XGBRegressor Evaluation
MAE: 0.4217486854848762
MSE: 4.955320528455929
R^2 Score: 0.9673029525402662
MAPE: 19.472205161410127%
Prediction Accuracy: 80.52779483858987%
RandomForestRegressor Evaluation
MAE: 0.3924395133510718
MSE: 5.659966460786213
R^2 Score: 0.9626534366594256
MAPE: 16.274248115014696%
Prediction Accuracy: 83.7257518849853%
SVR Evaluation
MAE: 0.38247366525322757
MSE: 5.1045005255745215
R^2 Score: 0.966318607447387
MAPE: 19.99052543018962%
Prediction Accuracy: 80.00947456981038%


array([13.19205804, 25.00710307, 21.99223082, ..., -1.76695277,
       18.54828614, -3.25267725])

In [ ]:
# แสดงตัวอย่างข้อมูลที่แปลงแล้ว
print(data.head())
print(data.shape)


         date  temperature (°C)  humidity (%)  wind_speed (km/h)  \
0  1925-01-01         28.488867     63.548650          29.789863   
1  1925-01-02         28.992703     95.005496          25.575581   
2  1925-01-03         21.070264     99.813784           1.543178   
3  1925-01-04         32.380113     76.746228           4.649422   
4  1925-01-05               NaN     61.691885          31.451403   

   country_France  country_Japan  country_South Korea  country_Thailand  \
0               0              0                    0                 1   
1               0              0                    0                 1   
2               0              0                    0                 1   
3               0              0                    0                 1   
4               0              0                    0                 1   

   country_UK  country_USA  city_Beijing  city_London  city_New York  \
0           0            0             0            0              0

In [ ]:
# บันทึกโมเดล

joblib.dump(xgb, 'xgb_temp.pkl')
joblib.dump(xgb, 'rf_temp.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(svm, 'svm_temp.pkl')


['svm_temp.pkl']

In [ ]:
# โหลดโมเดลที่เทรนไว้
model_1 = joblib.load('xgb_temp.pkl')
model_2 = joblib.load('rf_temp.pkl')
model_3 = joblib.load('svm_temp.pkl')
scaler = joblib.load('scaler.pkl')

In [ ]:


# โหลดโมเดลที่เคยเทรนไว้
model_1 = joblib.load('xgb_temp.pkl')  # SVM
model_2 = joblib.load('rf_temp.pkl')   # Random Forest
model_3 = joblib.load('svm_temp.pkl')  # XGBoost

# โหลด Scaler (ถ้ามีการใช้ Normalize หรือ Standardize)
scaler = joblib.load('scaler.pkl')

# สร้าง Voting Regressor โดยใช้โมเดลที่โหลดมา
ensemble_model = VotingRegressor(estimators=[
    ('svm', model_1),
    ('random_forest', model_2),
    ('xgb', model_3)
])

# ฝึกสอน Voting Regressor (ถ้าต้องการ Re-train)
ensemble_model.fit(X_train, y_train_temp)

# ประเมินผลโมเดล
evaluate_model(ensemble_model, X_test, y_test_temp)

#บันทึกโมเดลที่ Train แล้ว
joblib.dump(ensemble_model, 'ensemble_model_temp.pkl')

# โหลดโมเดลที่บันทึกไว้
ensemble_model_loaded = joblib.load('ensemble_model_temp.pkl')

# ทำนายผลจากโมเดลที่โหลด
y_pred_ensemble = ensemble_model_loaded.predict(X_test)


VotingRegressor Evaluation
MAE: 0.39929091299764785
MSE: 4.956862931889015
R^2 Score: 0.9672927751888791
MAPE: 17.510562241145067%
Prediction Accuracy: 82.48943775885493%


In [ ]:
# Define a custom evaluation function for ensemble model
def evaluate_ensemble_model(model, X_test, y_test):
    y_pred_ensemble = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred_ensemble)
    mse = mean_squared_error(y_test, y_pred_ensemble)
    r2 = r2_score(y_test, y_pred_ensemble)
    mape = np.mean(np.abs((y_test - y_pred_ensemble) / y_test)) * 100
    accuracy = 100 - mape

    print(f"{model.__class__.__name__} Evaluation")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"R^2 Score: {r2}")
    print(f"MAPE: {mape}%")
    print(f"Prediction Accuracy: {accuracy}%")
    print("=" * 50)
    return y_pred_ensemble

# Evaluate the ensemble model
evaluate_ensemble_model(ensemble_model, X_test, y_test_temp)


VotingRegressor Evaluation
MAE: 0.1867406581142742
MSE: 0.8237213278685923
R^2 Score: 0.9565919165288196
MAPE: 0.7040708528027145%
Prediction Accuracy: 99.29592914719728%


array([34.12147558, 32.9040334 , 22.45897408, ..., 26.61676334,
       29.51993897, 32.62038653])